In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import bs4 

In [3]:
import os

In [4]:
os.getcwd()

'/media/ubuntu/disk'

In [8]:
os.chdir('disk')

In [6]:
os.chdir("..")

In [5]:
df_LR_HR = pd.read_csv('LR_HR_pairs.csv')
df_LR_HR.columns

Index(['Unnamed: 0', 'ohrc', 'tmc'], dtype='object')

In [ ]:
i = 371
while i < len(df_LR_HR):
    try:
        ohrc = df_LR_HR.iloc[i]['ohrc']
        tmc = df_LR_HR.iloc[i]['tmc']

        os.chdir('ohrc_calibrated/')
        os.chdir(ohrc+'/data/calibrated/'+ohrc[12:20])
#         print(1,os.getcwd())

        lat = np.zeros((2, 2))
        lon = np.zeros((2, 2))
        for file in os.listdir():
            if '.xml' in file:
                with open(file, 'r') as xml_file:
                    contents = xml_file.read()

                soup = bs4.BeautifulSoup(contents, 'lxml')
                lat[0, 0] = float(soup.find_all('isda:upper_left_latitude')[1].text)
                lat[0, 1] = float(soup.find_all('isda:upper_right_latitude')[1].text)
                lat[1, 0] = float(soup.find_all('isda:lower_left_latitude')[1].text)
                lat[1, 1] = float(soup.find_all('isda:lower_right_latitude')[1].text)

                lon[0, 0] = float(soup.find_all('isda:upper_left_longitude')[1].text)
                lon[0, 1] = float(soup.find_all('isda:upper_right_longitude')[1].text)
                lon[1, 0] = float(soup.find_all('isda:lower_left_longitude')[1].text)
                lon[1, 1] = float(soup.find_all('isda:lower_right_longitude')[1].text)

        os.chdir('../../../../../')
#         print(2,os.getcwd())

        os.chdir('tmc_intersects/')
        print(3,os.getcwd())
        target_tmc = ''
        for tmc_file in os.listdir():
            if tmc in tmc_file:
                target_tmc = tmc_file

        os.chdir(target_tmc+'/geometry/calibrated/'+target_tmc[12:20]) #tmc file's csv coordinates
#         print(4,os.getcwd())
        for file in os.listdir():
            if '.csv' in file:
                df = pd.read_csv(file)

        df['ul_lon_difference'] = df['Longitude'] - lon[0, 0]
        df['ur_lon_difference'] = df['Longitude'] - lon[0, 1]
        df['ll_lon_difference'] = df['Longitude'] - lon[1, 0]
        df['lr_lon_difference'] = df['Longitude'] - lon[1, 1]

        df['ul_lat_difference'] = df['Latitude'] - lat[0, 0]
        df['ur_lat_difference'] = df['Latitude'] - lat[0, 1]
        df['ll_lat_difference'] = df['Latitude'] - lat[1, 0]
        df['lr_lat_difference'] = df['Latitude'] - lat[1, 1]     

#         print(5,os.getcwd())
        os.chdir('../../../')
        print(6,os.getcwd())
        os.chdir('data/calibrated/'+target_tmc[12:20])

        w = 0
        h = 0
        for file in os.listdir():
            if '.xml' in file:
                with open(file, 'r') as xml_file:
                    contents = xml_file.read()
                soup = bs4.BeautifulSoup(contents, 'lxml')
                w=int(soup.find_all("elements")[0].text)
                h=int(soup.find_all("elements")[1].text)

                data_type = soup.find_all('data_type')[0].text

        for file in os.listdir():
            if '.img' in file:
                with open(file, 'rb') as img:
                    img_bytes = img.read()

                    if data_type == 'UnsignedByte':
                        tmc_img_array = np.frombuffer(img_bytes, dtype = np.uint8).reshape(w, h)
                    else:
                        tmc_img_array = np.frombuffer(img_bytes, dtype = np.uint16).reshape(w, h)

        ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])
        ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])
        ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])
        ll_lat_scan=int(df[abs(df['ll_lat_difference']) == min(abs(df['ll_lat_difference']))]['Scan'])

        if((ur_lat_scan> ll_lat_scan) and (ur_long_pix< ll_long_pix)):
            plt.imshow(tmc_img_array[ll_lat_scan:ur_lat_scan, ur_long_pix:ll_long_pix], cmap='gray', origin='lower')
            print(1)
        elif((ur_lat_scan> ll_lat_scan) and (ur_long_pix> ll_long_pix)):
            plt.imshow(tmc_img_array[ll_lat_scan:ur_lat_scan, ll_long_pix:ur_long_pix], cmap='gray', origin='lower')
            print(2)
        elif((ur_lat_scan< ll_lat_scan) and (ur_long_pix< ll_long_pix)):
            plt.imshow(tmc_img_array[ur_lat_scan:ll_lat_scan, ur_long_pix:ll_long_pix], cmap='gray', origin='lower')
            print(3)
        elif((ur_lat_scan< ll_lat_scan) and (ur_long_pix > ll_long_pix)):
            plt.imshow(tmc_img_array[ur_lat_scan:ll_lat_scan, ll_long_pix:ur_long_pix], cmap='gray', origin='lower')
            print(4)

        os.chdir('../../../../../')
        print(7,os.getcwd())
        plt.axis('off')
        plt.savefig('saved_tmc_cropping/'+str(i),pad_inches=0,bbox_inches='tight' )
        i += 1
        
    except Exception as e:
        os.chdir('../../../../../')
        print(7,os.getcwd())
        print('lol')
        print(e)
        i += 1

3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220109T1615439830_d_img_d32


/tmp/ipykernel_8092/1678267675.py:82: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])
/tmp/ipykernel_8092/1678267675.py:83: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])
/tmp/ipykernel_8092/1678267675.py:84: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])
/tmp/ipykernel_8092/1678267675.py:85: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

1
7 /media/ubuntu/disk
3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220109T1022082458_d_img_d32


/tmp/ipykernel_8092/1678267675.py:82: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])
/tmp/ipykernel_8092/1678267675.py:83: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])
/tmp/ipykernel_8092/1678267675.py:84: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])
/tmp/ipykernel_8092/1678267675.py:85: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

2
7 /media/ubuntu/disk
3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220109T1219587048_d_img_d32


...

2
7 /media/ubuntu/disk
3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220110T1352097832_d_img_d32


/tmp/ipykernel_8092/1678267675.py:82: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])
/tmp/ipykernel_8092/1678267675.py:83: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])
/tmp/ipykernel_8092/1678267675.py:84: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])
/tmp/ipykernel_8092/1678267675.py:85: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

2
7 /media/ubuntu/disk
3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220110T0956254489_d_img_d32


/tmp/ipykernel_8092/1678267675.py:82: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])
/tmp/ipykernel_8092/1678267675.py:83: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])
/tmp/ipykernel_8092/1678267675.py:84: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])
/tmp/ipykernel_8092/1678267675.py:85: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

1
7 /media/ubuntu/disk
3 /media/ubuntu/disk/tmc_intersects
6 /media/ubuntu/disk/tmc_intersects/ch2_tmc_ncn_20220110T1154169263_d_img_d32


In [ ]:
with open('ch2_ohr_ncp_20190906T1246532096_d_img_d18.xml', 'r') as xml_file:
    contents = xml_file.read()

soup = bs4.BeautifulSoup(contents, 'lxml')

In [ ]:
lat = np.zeros((2, 2))
lon = np.zeros((2, 2))

In [ ]:
lat[0, 0] = float(soup.find_all('isda:upper_left_latitude')[1].text)
lat[0, 1] = float(soup.find_all('isda:upper_right_latitude')[1].text)
lat[1, 0] = float(soup.find_all('isda:lower_left_latitude')[1].text)
lat[1, 1] = float(soup.find_all('isda:lower_right_latitude')[1].text)
lat

In [ ]:
lon[0, 0] = float(soup.find_all('isda:upper_left_longitude')[1].text)
lon[0, 1] = float(soup.find_all('isda:upper_right_longitude')[1].text)
lon[1, 0] = float(soup.find_all('isda:lower_left_longitude')[1].text)
lon[1, 1] = float(soup.find_all('isda:lower_right_longitude')[1].text)
lon

In [ ]:
csv = 'ch2_tmc_ncn_20230205T1606344957_g_grd_n18.csv'

In [ ]:
df = pd.read_csv(csv)
df['ul_lon_difference'] = df['Longitude'] - lon[0, 0]
df['ur_lon_difference'] = df['Longitude'] - lon[0, 1]
df['ll_lon_difference'] = df['Longitude'] - lon[1, 0]
df['lr_lon_difference'] = df['Longitude'] - lon[1, 1]

df['ul_lat_difference'] = df['Latitude'] - lat[0, 0]
df['ur_lat_difference'] = df['Latitude'] - lat[0, 1]
df['ll_lat_difference'] = df['Latitude'] - lat[1, 0]
df['lr_lat_difference'] = df['Latitude'] - lat[1, 1]

In [ ]:
df

In [ ]:
ul_lon_df = df[abs(df['ul_lon_difference']) < 0.1]
ur_lon_df = df[abs(df['ur_lon_difference']) < 0.1]
ll_lon_df = df[abs(df['ll_lon_difference']) < 0.1]
lr_lon_df = df[abs(df['lr_lon_difference']) < 0.1]

ul_lat_df = df[abs(df['ul_lat_difference']) < 0.1]
ur_lat_df = df[abs(df['ur_lat_difference']) < 0.1]
ll_lat_df = df[abs(df['ll_lat_difference']) < 0.1]
lr_lat_df = df[abs(df['lr_lat_difference']) < 0.1]

In [ ]:
ur_long_pix=int(df[abs(df['ur_lon_difference']) == min(abs(df['ur_lon_difference']))]['Pixel'])

In [ ]:
ll_long_pix=int(df[abs(df['ll_lon_difference']) == min(abs(df['ll_lon_difference']))]["Pixel"])

In [ ]:
ur_lat_scan=int(df[abs(df['ur_lat_difference']) == min(abs(df['ur_lat_difference']))]["Scan"])

In [ ]:
ll_lat_scan=int(df[abs(df['ll_lat_difference']) == min(abs(df['ll_lat_difference']))]['Scan'])

In [ ]:
with open('ch2_tmc_ncn_20230205T1606344957_d_img_n18.xml', 'r') as xml_file:
    contents = xml_file.read()

soup = bs4.BeautifulSoup(contents, 'lxml')

In [ ]:
w=int(soup.find_all("elements")[0].text)
h=int(soup.find_all("elements")[1].text)

In [ ]:
filename = 'ch2_tmc_ncn_20230205T1606344957_d_img_n18.img' 

# set width and height 
# w, h = 176476, 4000 

with open(filename, 'rb') as img:
    img_bytes = img.read()
len(img_bytes)

In [ ]:
img_array = np.frombuffer(img_bytes, dtype = np.uint16).reshape(w, h)
img_array.shape

In [ ]:
print(ur_lat_scan,ll_lat_scan)

In [ ]:
if((ur_lat_scan> ll_lat_scan) and (ur_long_pix< ll_long_pix)):
    plt.imshow(img_array[ll_lat_scan:ur_lat_scan, ur_long_pix:ll_long_pix], cmap='gray', origin='lower')
    print(1)
elif((ur_lat_scan> ll_lat_scan) and (ur_long_pix> ll_long_pix)):
    plt.imshow(img_array[ll_lat_scan:ur_lat_scan, ll_long_pix:ur_long_pix], cmap='gray', origin='lower')
    print(2)
elif((ur_lat_scan< ll_lat_scan) and (ur_long_pix< ll_long_pix)):
    plt.imshow(img_array[ur_lat_scan:ll_lat_scan, ur_long_pix:ll_long_pix], cmap='gray', origin='lower')
    print(3)
elif((ur_lat_scan< ll_lat_scan) and (ur_long_pix > ll_long_pix)):
    plt.imshow(img_array[ur_lat_scan:ll_lat_scan, ll_long_pix:ur_long_pix], cmap='gray', origin='lower')
    print(4)

another image

In [ ]:
filename = 'ch2_tmc_ncn_20211111T1322553940_d_img_d18.img' 

# set width and height 
w, h = 190796, 4000 

with open(filename, 'rb') as img:
    img_bytes = img.read()
len(img_bytes)

In [ ]:
img_array = np.frombuffer(img_bytes, dtype = np.uint16).reshape(w, h)
img_array.shape

In [ ]:
plt.imshow(img_array[185600:180600:-1, 500:1200], cmap='gray', origin='lower')